# Experiment Analysis

### Plot Training Curves

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

################################
# SELECT MODELS ################
################################
model_names = ["daflowgnn-2"]
props = ["gain"]
runs = [1]
###############################
###############################
###############################

plt.figure(figsize=(12, 6))
plt.grid()
plt.xlabel("Epochs")
plt.ylabel("RMSE")

for model, prop, run in zip(model_names, props, runs):
    
    path = f"./../experiments/supervised/{model}/{prop}/run_{run}/"
    results = pd.read_csv(path + "results.csv")
    results = results[results["train_rmse"] > 0]

    plt.plot(results["train_rmse"], label=f"{model}_{prop}_{run}_train")
    plt.plot(results["val_rmse"], label=f"{model}_{prop}_{run}_val")
    plt.plot(results["test_rmse"], label=f"{model}_{prop}_{run}_test")

plt.legend()
plt.show()

### Display Leaderboard

In [ ]:
import os
import numpy as np
import pandas as pd

from IPython.display import display, HTML

################################
# SELECT MODELS ################
################################
models = ["flowgat", "flowgatv2", "flowtransformer", "daflowgnn-1", "daflowgnn-2"]
props = ["gain", "bw", "fom"]
################################
################################
################################

leaderboard = np.zeros((len(models), len(props)*2))
patience = 20

leaderboard_models = []
for i, model in enumerate(models):

    model_dir = f"./../experiments/supervised/{model}/"
    if not os.path.exists(model_dir):
        print(f"{model} does not exist!")
        continue
    leaderboard_models.append(model)
    
    for j, prop in enumerate(props):

        prop_dir = model_dir + f"{prop}/"
        if not os.path.exists(prop_dir):
            continue

        scores = []
        for run in range(1, 11):
            run_dir = prop_dir + f"run_{run}/"
            if not os.path.exists(run_dir):
                continue
                
            results = pd.read_csv(run_dir + "results.csv")
            results = results[results["train_rmse"] > 0]
            scores.append(results["test_rmse"].values[-(patience + 1)])

        mean, std = np.mean(scores), np.std(scores)
        leaderboard[i, 2 * j] = mean
        leaderboard[i, 2 * j + 1] = std

leaderboard = leaderboard[np.any(leaderboard > 0, axis=1)]
leaderboard = np.round(leaderboard, 3)
columns = []
for prop in props:
    columns.append(f"{prop}_mean")
    columns.append(f"{prop}_std")
    
leaderboard = pd.DataFrame(leaderboard, columns=columns, index=leaderboard_models)
if len(props) > 1:
    leaderboard["mean_score"] = np.round(leaderboard[[prop + "_mean" for prop in props]].mean(axis=1), 3)
    leaderboard = leaderboard.sort_values(by="mean_score")
else:
    leaderboard = leaderboard.sort_values(by=props[0] + "_mean")
    
leaderboard = leaderboard.reset_index()
leaderboard.index += 1
leaderboard = leaderboard.rename(columns={"index": "model"})

display(HTML(leaderboard.to_html()))
